# POAFeatures Class
The POA features class will take the output of a multiple-sequence alignment and tie IPD and PW to each individual base. At a later date, other by-base features can be tied to each base.  

This notebook will serve as a prototype.

In [65]:
import POA as poa
from pbcore.io import (SubreadSet,
                       ReferenceSet)
import pandas as pd
import numpy as np

In [4]:
# initialize dataset paths
sset_path = '/pbi/dept/itg/zia/projects/3509/conditions/100nM/subreads/input.subreadset.xml'
rset_path = '/pbi/dept/itg/zia/projects/3509/conditions/100nM/reference.fasta'
sset = SubreadSet(sset_path)
rset = ReferenceSet(rset_path)

In [30]:
# grab one ZMWs subreads
pbi_df = pd.DataFrame.from_records(sset.index)
pbi_gb = pbi_df.groupby(by=['holeNumber'])
for group in pbi_gb:
    zmw = group[0]
    pbi_subreads_df = group[1]
    subread_ixs = list(pbi_subreads_df.index)
    subreads = [subread for subread in sset[subread_ixs] \
                if subread.contextFlag == 3] # ensure adapters flank both ends
    break

In [113]:
class PoaWithFeatures(poa.POA):
    """
    Takes as input adapter-flanked subreads from
    a particular ZMW, produces a multiple-sequence
    alignment using partial-order graphs, and ties
    in IPD and PW info to each raw subread.
    """
    def __init__(self, subreads):
        self.subreads = subreads
        self.subread_names = np.array([s.readName for s in self.subreads])
        self.POA = poa.POA(self.subreads) # generate POA object
        self.PoaGraph = self.POA.generatePoaGraph() # perform POA MSA
        self.PoaStrings = self.PoaGraph.generateAlignmentStrings() # convert graph to strings
        self.MSAs = self.PoaStrings[0:-1]
        self.PluralityConsensus = self.PoaStrings[-1]
        self.foldInMsaFeatures = self.foldInFeatures()
        
    def foldInFeatures(self):
        """
        For each alignment, connect the by-base features.
        Each deleted base will have 0 stored for each feature.
        By-base features include IPD and PW.
        """
        feature_vector = []
        for msa in self.MSAs:
            read_name = msa[0]
            sequence = msa[1]
            base_ixs = np.flatnonzero(np.array(list(sequence)) != '-')
            raw_subread_ix = np.flatnonzero(self.subread_names == read_name)[0]
            subread = self.subreads[raw_subread_ix]
            features = np.zeros((len(sequence), ), dtype=[('IPD', int),
                                                          ('PW', int)])
            features['IPD'][base_ixs] = subread.IPD(aligned=False)
            features['PW'][base_ixs] = subread.PulseWidth(aligned=False)
            feature_vector.append((read_name, features))
        return feature_vector
        

In [114]:
pwf = PoaWithFeatures(subreads)

In [115]:
pwf.foldInMsaFeatures

[('m54004_180305_005953/4522450/2024_2155',
  array([(  0,  0), (  0,  0), (  0,  0), (  0,  0), (101,  4), (  0,  0),
         (  0,  0), (  5,  4), (  0,  0), (  0,  0), (  0,  0), (  0,  0),
         (103,  8), (  0,  0), (  0,  0), (  0,  0), ( 26,  7), ( 34,  7),
         ( 34,  9), (  8,  3), (118,  6), ( 57,  5), ( 42, 12), (  7, 61),
         ( 90,  9), (  0,  0), ( 34, 17), (322, 67), (  0,  0), ( 40,  6),
         ( 13,  8), ( 28,  6), ( 25,  2), ( 65, 13), ( 15, 23), (  0,  0),
         ( 16, 32), ( 87,  4), ( 24, 21), ( 48, 16), ( 89,  6), (105, 22),
         ( 23, 15), ( 92, 11), ( 96,  4), ( 67, 44), ( 66, 11), (  9,  5),
         (179, 14), ( 21, 53), ( 10,  3), ( 52,  4), ( 92, 37), ( 26, 27),
         (  0,  0), ( 14, 17), (  0,  0), (  6,  8), ( 64,  4), ( 42,  5),
         (  0,  0), (  0,  5), (  3, 16), ( 78, 27), ( 13,  9), (142, 15),
         (  0,  0), ( 36,  8), (  0,  0), (  7, 58), ( 13, 10), (  0,  0),
         (  5, 11), ( 68,  8), (  9, 20), ( 23,  9), (  

In [112]:
tmp

[('m54004_180305_005953/4522450/2024_2155',
  array([(  0,  0), (  0,  0), (  0,  0), (  0,  0), (101,  4), (  0,  0),
         (  0,  0), (  5,  4), (  0,  0), (  0,  0), (  0,  0), (  0,  0),
         (103,  8), (  0,  0), (  0,  0), (  0,  0), ( 26,  7), ( 34,  7),
         ( 34,  9), (  8,  3), (118,  6), ( 57,  5), ( 42, 12), (  7, 61),
         ( 90,  9), (  0,  0), ( 34, 17), (322, 67), (  0,  0), ( 40,  6),
         ( 13,  8), ( 28,  6), ( 25,  2), ( 65, 13), ( 15, 23), (  0,  0),
         ( 16, 32), ( 87,  4), ( 24, 21), ( 48, 16), ( 89,  6), (105, 22),
         ( 23, 15), ( 92, 11), ( 96,  4), ( 67, 44), ( 66, 11), (  9,  5),
         (179, 14), ( 21, 53), ( 10,  3), ( 52,  4), ( 92, 37), ( 26, 27),
         (  0,  0), ( 14, 17), (  0,  0), (  6,  8), ( 64,  4), ( 42,  5),
         (  0,  0), (  0,  5), (  3, 16), ( 78, 27), ( 13,  9), (142, 15),
         (  0,  0), ( 36,  8), (  0,  0), (  7, 58), ( 13, 10), (  0,  0),
         (  5, 11), ( 68,  8), (  9, 20), ( 23,  9), (  